## Visualização e pré-processamento

A seguir, a implementação de cada um dos filtros: passa-faixa (bandpass), passa-baixa (lowpass), passa-alta (highpass) e corta-faixa (notch).

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
from scipy import signal
from sklearn.preprocessing import LabelEncoder
import mne

Para imprimir os dados no domínio da frequência, foi utilizado a função `matplotlib.pyplot.psd` que na verdade utiliza a transformação dos dados por FFT, porém aplica uma característica que evidencia a "energia" dos dados, chamado de PSD (_Power Spectrum Density_)

Definição de função para imprimir os dados no domínio do tempo e da frequência:

In [2]:
def print_graphs(data):
    for i in range(0, 21, 3):
        plt.plot(data[i,:])
    plt.title('Domínio do tempo')
    plt.show()

    for i in range(0, 21, 3):
        plt.psd(data[i,:], Fs=512)
    plt.title('Domínio da frequência')
    plt.show()

    for i in range(0, 21, 3):
        plt.specgram(data[i,:], Fs=512)
    plt.title('Domínio da frequência')
    plt.show()

Carregamento dos dados de um participante da base de dados AVI:

In [13]:
data = np.load("../datasets/avi/single/data.npy")
print(data.shape)

# carregando somente 1 participante
data = data[0]
print(data.shape)

(4, 21, 15360)
(21, 15360)


Visualização do sinal em todos os domínios apresentados, sem a aplicação de nenhum filtro:

In [ ]:
rcParams['figure.figsize'] = [16., 5.]
print_graphs(data)

Para facilitar o trabalho de visualização dos dados nas faixas de frequência de trabalho, vamos utilizar a ferramenta `MNE`:

In [5]:
# criação de um objeto "info"
n_channels = 1
sfreq = 512
ch_names = ['Oz']
ch_types = ['eeg']
info = mne.create_info(ch_names, sfreq=sfreq, ch_types=ch_types)
info

<Info | 7 non-empty values
 bads: []
 ch_names: Oz
 chs: 1 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 256.0 Hz
 meas_date: unspecified
 nchan: 1
 projs: []
 sfreq: 512.0 Hz
>

In [14]:
# data : array, shape (n_epochs, n_channels, n_times)
data_correct = data.reshape(data.shape[0], 1, data.shape[1])
print(data_correct.shape)

# adicionando label como "épocas" do sinal
labels = np.load("../datasets/avi/single/labels.npy")[0]
# print(labels, labels.shape)

# método para transformar labels categóricos
le = LabelEncoder()
events = np.column_stack((
    np.array(range(len(labels))),
    np.zeros(21, dtype=int),
    le.fit_transform(labels))
)
print(events)

event_dict = {'6':0, '6.5':1, '7':2, '7.5':3, '8.2':4, '9.3':5, '10':6}
mne_data = mne.EpochsArray(data_correct, info, events, event_id=event_dict)
print(mne_data)

(21, 1, 15360)
[[ 0  0  6]
 [ 1  0  6]
 [ 2  0  6]
 [ 3  0  1]
 [ 4  0  1]
 [ 5  0  1]
 [ 6  0  0]
 [ 7  0  0]
 [ 8  0  0]
 [ 9  0  3]
 [10  0  3]
 [11  0  3]
 [12  0  2]
 [13  0  2]
 [14  0  2]
 [15  0  4]
 [16  0  4]
 [17  0  4]
 [18  0  5]
 [19  0  5]
 [20  0  5]]
Not setting metadata
21 matching events found
No baseline correction applied
0 projection items activated


Number of events,21
Events,10: 36: 36.5: 37: 37.5: 38.2: 39.3: 3
Time range,0.000 – 29.998 s
Baseline,off


In [ ]:
import warnings
warnings.filterwarnings('ignore')

# view = mne_data.compute_psd(method='welch', fmin=3, fmax=13)
for i in range(len(mne_data)):
    view = mne_data[i].compute_psd(method='multitaper', fmin=3, fmax=13, verbose=False)
    view.plot(show=False)
    print()
    plt.title('Domínio da frequência')
    plt.axvline(x=float(list(mne_data[i].event_id)[0]), linestyle='--', color='green')
    plt.show()
print()


In [8]:
mne_data.filter( 4, 14, fir_design='firwin')

Setting up band-pass filter from 4 - 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 14.00 Hz
- Upper transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 15.75 Hz)
- Filter length: 845 samples (1.650 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Number of events,21
Events,10: 36: 36.5: 37: 37.5: 38.2: 39.3: 3
Time range,0.000 – 29.998 s
Baseline,off


In [9]:
mne_data.save('../datasets/avi/single/filtered-3-13-sub-1.fif', overwrite=True)

Overwriting existing file.


### Tarefas

- Estudar e utilizar os exemplos já apresentados no repositório `tutoriais` e criar o objeto `mne.EpochArray` para os dados *single* e *multi-target* (`AVI dataset`);
- Estudar documentos MNE para realizar a aplicação dos filtros temporais a partir de objetos `mne.EpochArray` (sempre verificando as frequências utilizadas como targets dos `datasets`). Isto também para os dados *single* e *multi-target* (`AVI dataset`);
- **Ótimo**: elaborar toda a aplicação já realizada até o ponto anterior para o *dataset* `Beta`.

##### Single Target

In [10]:
data = np.load("../datasets/avi/single/data.npy")



count = 0
for subject in data:
    data_correct = subject.reshape(subject.shape[0], 1, subject.shape[1])
    print(data_correct.shape)
    mne_d = mne.EpochsArray(data_correct, info, events, event_id=event_dict)
    mne_data.filter( 4, 14, fir_design='firwin')
    mne_data.save('../datasets/avi/single/filtered-4-14-sub-'+str(count)+'.fif', overwrite=True)
    count += 1

(21, 1, 15360)
Not setting metadata
21 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 4 - 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 14.00 Hz
- Upper transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 15.75 Hz)
- Filter length: 845 samples (1.650 s)

Overwriting existing file.
(21, 1, 15360)
Not setting metadata
21 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 4 - 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window w

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


21 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 4 - 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 14.00 Hz
- Upper transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 15.75 Hz)
- Filter length: 845 samples (1.650 s)



Overwriting existing file.
(21, 1, 15360)
Not setting metadata
21 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 4 - 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 14.00 Hz
- Upper transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 15.75 Hz)
- Filter length: 845 samples (1.650 s)

Overwriting existing file.


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


##### Multi Target

In [11]:
data = np.load("../datasets/avi/multi/data.npy")
labels = np.load("../datasets/avi/multi/labels.npy")
event_dict = {'6':0, '6.5':1, '7':2, '7.5':3, '8.2':4, '9.3':5}

for i in range(len(data)):
    data_correct = data[i].reshape(data[i].shape[0], 1, data[i].shape[1])
    print(data_correct.shape)
    le = LabelEncoder()
    events = np.column_stack((
        np.array(range(len(labels[i]))),
        np.zeros(20, dtype=int),
        le.fit_transform(labels[i]))
    )
    mne_d = mne.EpochsArray(data_correct, info, events, event_id=event_dict)
    mne_data.filter( 4, 14, fir_design='firwin')
    mne_data.save('../datasets/avi/multi/filtered-4-14-sub-'+str(i)+'.fif', overwrite=True)


(20, 1, 8192)
Not setting metadata
20 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 4 - 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 14.00 Hz
- Upper transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 15.75 Hz)
- Filter length: 845 samples (1.650 s)

Overwriting existing file.
(20, 1, 8192)
Not setting metadata
20 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 4 - 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window wit

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


(20, 1, 8192)
Not setting metadata
20 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 4 - 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 14.00 Hz
- Upper transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 15.75 Hz)
- Filter length: 845 samples (1.650 s)

Overwriting existing file.
(20, 1, 8192)
Not setting metadata
20 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 4 - 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window wit

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
